# Azure AI Agent with Code Interpreter Example

This notebook demonstrates using HostedCodeInterpreterTool with Azure AI Agents for Python code execution and mathematical problem solving.

## Features Covered:
- Setting up a code interpreter tool for agents
- Executing Python code through the agent
- Mathematical problem solving
- Accessing code interpreter inputs and outputs
- Working with streaming responses

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Azure AI Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with connection details
4. **Dependencies**: Required agent-framework packages installed

If you need to use a different tenant, specify the tenant ID:
```bash
az login --tenant <tenant-id>
```

## Import Libraries

Import the required libraries for Azure AI agent functionality.

In [ ]:
import os
from pathlib import Path
from agent_framework import AgentRunResponse, ChatResponseUpdate, HostedCodeInterpreterTool
from agent_framework.azure import AzureAIAgentClient
from azure.ai.agents.models import (
    RunStepDeltaCodeInterpreterDetailItemObject,
)
from azure.identity import InteractiveBrowserCredential, AzureCliCredential

## Initial Setup

Initialize the Azure AI project client and set up authentication.

In [ ]:
from pathlib import Path  # For working with file paths
import os  # For environment variables
import time  # For sleep function
from dotenv import load_dotenv  # For loading environment variables from .env file
from azure.identity import InteractiveBrowserCredential, AzureCliCredential  # For Azure authentication
from azure.ai.projects import AIProjectClient

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv('../../../.env')  # Load environment variables from .env file

# Get connection string and tenant ID for authentication
conn_string = os.environ.get("PROJECT_CONNECTION_STRING")
tenant_id = os.environ.get("TENANT_ID")

print(f"🔑 Using Tenant ID: {tenant_id}")
print(f"🔗 Using Connection String: {conn_string[:50]}..." if conn_string else "❌ No connection string found")

# Initialize the AI Project Client using PROJECT_CONNECTION_STRING
try:
    print("🌐 Using browser-based authentication...")
    
    # Use InteractiveBrowserCredential with fallback to AzureCliCredential
    credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    
    # Create the project client using the same pattern as other working notebooks
    # We'll use the conn_string directly in the AzureAIAgentClient
    print("✅ Authentication setup completed")
except Exception as e:
    # Print error message if client initialization fails
    print(f"❌ Error in authentication setup: {str(e)}")
    print("💡 Please complete the browser authentication prompt that should appear")

## Helper Function for Code Interpreter Data

This helper function allows us to access and display the actual code that was executed by the code interpreter:

In [ ]:
def print_code_interpreter_inputs(response) -> None:
    """Helper method to access code interpreter data."""
    
    print("\nCode Interpreter Inputs during the run:")
    
    # Handle different response types
    if hasattr(response, 'raw_representation') and response.raw_representation is not None:
        for chunk in response.raw_representation:
            if isinstance(chunk, ChatResponseUpdate) and isinstance(
                chunk.raw_representation, RunStepDeltaCodeInterpreterDetailItemObject
            ):
                print(chunk.raw_representation.input, end="")
    else:
        print("No code interpreter inputs found in this response.")
    print("\n")

## Create and Run the Code Interpreter Agent

Now let's create an agent that can write and execute Python code to solve problems:

In [ ]:
async def main() -> None:
    """Example showing how to use the HostedCodeInterpreterTool with Azure AI."""
    print("=== Azure AI Agent with Code Interpreter Example ===")

    # Use the authentication pattern that works with PROJECT_CONNECTION_STRING
    credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    
    async with AzureAIAgentClient(
        project_endpoint=conn_string,
        async_credential=credential,
        model_deployment_name="gpt-4o"
    ) as chat_client:
        agent = chat_client.create_agent(
            name="CodingAgent",
            instructions=(
                "You are a helpful assistant that can write and execute Python code to solve problems."
            ),
            tools=HostedCodeInterpreterTool(),
        )
        query = "Generate the factorial of 100 using python code, show the code and execute it."
        print(f"User: {query}")
        response = await agent.run(query)
        print(f"Agent: {response}")
        # To review the code interpreter outputs, you can access
        # them from the response raw_representations, just uncomment the next line:
        # print_code_interpreter_inputs(response)

## Execute the Example

Run the main function to see the code interpreter in action:

In [ ]:
# Run the main function
await main()

## View Code Interpreter Inputs

Let's create another example that shows the actual code executed by the interpreter:

In [ ]:
async def detailed_example() -> None:
    """Example that shows code interpreter functionality."""
    print("=== Detailed Code Interpreter Example ===")

    credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    
    async with AzureAIAgentClient(
        project_endpoint=conn_string,
        async_credential=credential,
        model_deployment_name="gpt-4o"
    ) as chat_client:
        agent = chat_client.create_agent(
            name="DetailedCodingAgent",
            instructions=(
                "You are a helpful assistant that can write and execute Python code to solve problems. "
                "Always show your code and explain your approach."
            ),
            tools=HostedCodeInterpreterTool(),
        )
        
        # Use a clear mathematical query
        query = "Calculate the sum of squares of the first 10 numbers using Python code. Show the calculation step by step."
        print(f"User: {query}")
        
        try:
            response = await agent.run(query)
            print(f"Agent: {response}")
            print("✅ Code interpreter executed successfully!")
            
        except Exception as e:
            print(f"❌ Error occurred: {str(e)}")
            print("💡 This might be due to network connectivity issues. Try running again.")

# Run the detailed example
await detailed_example()

## Try More Complex Examples

Let's test the agent with more complex programming tasks:

In [ ]:
async def complex_examples():
    """Test the agent with more complex programming tasks."""
    
    credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    
    async with AzureAIAgentClient(
        project_endpoint=conn_string,
        async_credential=credential,
        model_deployment_name="gpt-4o"
    ) as chat_client:
        agent = chat_client.create_agent(
            name="AdvancedCodingAgent",
            instructions=(
                "You are an expert Python programmer. Write clean, efficient code and explain your solutions. "
                "Always show the Python code you execute."
            ),
            tools=HostedCodeInterpreterTool(),
        )
        
        # Use simpler queries to avoid network issues
        queries = [
            "Calculate the first 10 Fibonacci numbers using Python code",
            "Create a simple statistical analysis of the numbers [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",
            "Solve the quadratic equation x² - 5x + 6 = 0 using Python"
        ]
        
        for i, query in enumerate(queries, 1):
            print(f"\n--- Example {i}: {query} ---")
            try:
                response = await agent.run(query)
                print(f"Agent: {response}")
                print("✅ Example completed successfully!")
            except Exception as e:
                print(f"❌ Error in example {i}: {str(e)}")
                print("💡 Continuing with next example...")
            print("-" * 80)

# Run complex examples
await complex_examples()

## Key Takeaways

1. **Code Interpreter Tool**: The `HostedCodeInterpreterTool` allows agents to execute Python code in a secure environment
2. **Mathematical Computation**: Perfect for complex calculations like factorials, statistics, and mathematical operations
3. **Data Analysis**: Can perform data analysis tasks including generating plots and statistics
4. **Code Visibility**: You can access the actual code executed through `print_code_interpreter_inputs()`
5. **Streaming Support**: Works with both streaming and non-streaming responses
6. **Security**: Code execution happens in a secure, isolated environment
7. **Versatility**: Can handle various programming tasks from simple calculations to complex algorithms

## Best Practices

1. **Clear Instructions**: Provide clear instructions about what kind of code execution you need
2. **Error Handling**: The agent will handle code errors gracefully and suggest fixes
3. **Resource Management**: Use async context managers for proper cleanup
4. **Code Review**: Always review the executed code for understanding and validation
5. **Performance**: Be mindful of computational complexity for large-scale operations

## Use Cases

- Mathematical problem solving
- Data analysis and visualization
- Algorithm implementation and testing
- Educational coding assistance
- Prototype development
- Statistical computations

In [ ]:
# Let's test with a simpler example
async def simple_test():
    """Simple test to verify the code interpreter is working."""
    print("=== Simple Code Interpreter Test ===")
    
    credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    
    async with AzureAIAgentClient(
        project_endpoint=conn_string,
        async_credential=credential,
        model_deployment_name="gpt-4o"
    ) as chat_client:
        agent = chat_client.create_agent(
            name="SimpleTestAgent",
            instructions="You are a helpful assistant that can execute Python code.",
            tools=HostedCodeInterpreterTool(),
        )
        
        query = "Calculate 15 * 25 and print the result"
        print(f"User: {query}")
        response = await agent.run(query)
        print(f"Agent: {response}")

# Run simple test
await simple_test()